# Fitymi (Fake it till you make it)
#######################################
\# Jean-Paul van der Ham (22161392)
\# Versie 0.9, datum: 31-dec-2022.
#######################################

### Fitymi is een programma dat fake datasets genereert.


Voor de opdracht van ISDQ heb ik de volgende use-case bedacht:

```
Creëer met Fitymi een fake dataset voor de online Cryptobeurs: "BingoBit"

BingoBit verkoopt al crypto sinds 1970 op de Nederlandse markt.
Om crypto's te kunnen kopen zijn minimaal de volgende gegevens nodig: NAW-, beroeps-, contact- en bankgegevens.

Er is door het management gevraagd om voor de volgende enteiten het mogelijk te maken
een top x weer te geven in een barh of pie grafiek:

    1).  Voornaam
    2).  Achternaam
    3).  Beroep
    4).  Woonplaats
    5).  Provincie
    6).  CryptoNaam
    7).  AankoopJaar
```
Hieronder de code van Fitymi. Deze is opgedeeld in de volgende "hoofdstukken":
- Install & Import
- Genereer (faker)
- Manipuleer & Schrijf (pandas)
- Converteer & Open (openpyxl)
- Visualiseer (matplotlib)
- Help (github readme)
- Run

# Install & Import

Om het programma succesvol uit te voeren moet de volgende modules worden geïnstalleerd in python:

- pip install faker
- pip install pandas
- pip install matplotlib
- pip install openpyxl

En daarna worden geïmporteerd:

In [ ]:
from faker import Faker
import pandas as pd
import matplotlib.pyplot as plt
import os
import openpyxl
import webbrowser

# Genereer (faker)

De "genereer_klant_data" functie maakt gebruikt van faker (https://faker.readthedocs.io/en/master/locales/nl_NL.html#)

Deze functie verwacht twee "inputs":
1. land: Voor welk land genereer je de data (localization).
2. aantal: Hoeveel klanten (records) moeten er gegenereerd worden.

Er wordt een "python-list" gebruikt welke gevuld wordt door een "for-loop".

In [ ]:
def genereer_klant_data(land, aantal):

    # Bepaal voor welk land je data wil genereren (localization). Bijvoorbeeld: "nl_NL"
    fake = Faker(land)

    # Vul een lijst en geeft deze terug zodat hij buiten de functie gebruikt kan worden.
    klanten_lijst=[]
    for klanten in range(aantal):
        klanten_lijst.append([fake.first_name(), fake.last_name(), fake.date_of_birth(),fake.phone_number(), fake.job(), fake.company(), fake.ascii_email(), fake.ssn(), fake.iban(), fake.building_number(),fake.street_name(), fake.postcode(), fake.city(), fake.administrative_unit(), fake.cryptocurrency_name(),fake.pricetag(), fake.year(), fake.local_latlng(country_code = 'NL', coords_only = True)])
    return klanten_lijst

# Manipuleer & Schrijf (pandas)
Alle waardes gemaakt door "faker" zijn object strings. Voor bijvoorbeeld Matplotlib & PowerBI moeten er een aantal waardes worden omgezet naar int64 en datetime64.

Deze lijst wordt m.b.v. pandas getransformeerd naar een kolom / rij format en voorzien van kolom-namen. Daarna worden verschillende waardes omgezet naar andere datatypes.

Pandas schrijft deze daarna netjes weg in een xlsx-file op je je lokale schijf.

In [ ]:
def manipuleer_klant_data(klanten_lijst, naam):
    #Haal de huidige directory op
    working_directory=os.getcwd()
    bestand=naam+'.xlsx'
    # Maak een dataframe met kolom-namen
    kolom_namen = ['Voornaam', 'Achternaam', 'Geboortedatum','Telefoonnummer','Beroep','Bedrijf','EMail','BSNNummer','IBAN','Huisnummer','Straatnaam','Postcode','Woonplaats','Provincie','CryptoNaam', 'AankoopBedrag', 'AankoopJaar', 'Geo']
    df = pd.DataFrame(klanten_lijst,columns=kolom_namen)

    # Verwijder het euro-teken en de punt. Verander de punt in een komma.
    df['AankoopBedrag'] = df['AankoopBedrag'].str.replace('€','', regex=False)
    df['AankoopBedrag'] = df['AankoopBedrag'].str.replace('.','', regex=False)
    df['AankoopBedrag'] = df['AankoopBedrag'].str.replace(',','.', regex=False)

    # Verandert de waarde in de kolom Geboortedatum van string-object naar datetime
    df["Geboortedatum"] = pd.to_datetime(df["Geboortedatum"])

    # Faker geneert een tuple met coordinaten in de kolom Geo. Deze moet gesplits worden in twee nieuwe kolommen genaamd: longitude en latitude. Hiervoor is een kleine functie nodig.
    def split_coordinates(coordinates):
        latitude, longitude = coordinates
        return pd.Series([longitude, latitude])

    # Voeg de twee nieuwe kolommen toe.
    df[['Longitude', 'Latitude']] = df['Geo'].apply(split_coordinates)

    # Verwijder de kolom Geo
    df = df.drop(columns=['Geo'])

    # Verandert de waarde in de kolommen van string-object naar float64
    df[['BSNNummer', 'Huisnummer','AankoopBedrag','AankoopJaar', 'Longitude', 'Latitude']] = df[['BSNNummer', 'Huisnummer','AankoopBedrag','AankoopJaar', 'Longitude', 'Latitude']].apply(pd.to_numeric)

    # Schrijf het gemanipuleerde dataframe weg als xlsx
    try:
        df.to_excel(bestand, sheet_name=naam, index=False)
    except FileNotFoundError:
        print(f' Kan het bestand: {bestand} niet aanmaken.')

    print(f'Het bestand {naam}.xlsx kunt u hier vinden: {working_directory}')
    return df


# Converteer & Open (openpyxl)

Om het dataframe te kunnen gebruiken in PowerBI moet de xlsx worden geconverteerd naar een tabel-structuur.
Nadat deze is geconverteerd wordt excel gestart om dit te controleren.
Als alles er goed uit ziet kan je ervoor kiezen om gelijk online PowerBI te starten om deze dataset te importeren.

In [ ]:
def converteer_klant_data(df, naam):
    bestand=naam+'.xlsx'

    # Open het net gegenereerde xlsx bestand
    try:
        wb = openpyxl.load_workbook(bestand)
    except FileNotFoundError:
        print(f' Kan het bestand: {bestand} niet vinden.')

    # Selecteer alle rijen behalve de eerste rij (header-namen)
    tab = openpyxl.worksheet.table.Table(displayName=naam, ref=f'A1:{openpyxl.utils.get_column_letter(df.shape[1])}{len(df)+1}')

    # Maak van deze rijen een tabel
    wb[naam].add_table(tab)

    # Schrijf de xlsx weg als tabel en open deze in Excel om te controleren
    try:
        wb.save(bestand)
        os.system(f'open -a "Microsoft Excel" {bestand}')
    except FileNotFoundError:
        print(f' Kan het bestand: {bestand} niet overschrijven.')

    # Vraag of deze xlsx geïmporteerd moet worden in PowerBI online
    powerbi = input(f'Online PowerBI openen om {bestand} te importen? (j/n):')
    if powerbi == 'j':
        webbrowser.open('https://app.powerbi.com/groups/me/getdata/files/local-file')

# Visualiseer (matplotlib)
Deze functie kan over verschillende kolommen een top x visualiseren in de vorm van een barh of pie grafiek.
Via een menu wordt aan de gebruiker gevraagd welke top, welk type grafiek en over welke kolom.

In [ ]:
def visualiseer_klant_data(df):
    top = int(input('De top x (bijvoorbeeld 10 of 20)?:'))

    # Vraag welk type grafiek gebruikt moet worden.
    while True:
        grafiek = input('Wat voor type grafiek:'
                '''

                1).  barh
                2).  pie
                s).  stop
                ''')

        if grafiek == '1':
            grafiek_naam='barh'
            break
        if grafiek == '2':
            grafiek_naam='pie'
            break
        elif grafiek == 's':
            break
        else:
            print("Ongeldige keuze. Probeer het nog eens.")

    # Vraag welke kolom gebruik moet worden als bron.
    while True:
        kolom = input('Voor welke kolom?:'
                           """

                            1).  Voornaam
                            2).  Achternaam
                            3).  Beroep
                            4).  Woonplaats
                            5).  Provincie
                            6).  CryptoNaam
                            7).  AankoopJaar
                            s).  Stop
                             """)
        if kolom == '1':
            kolom_naam='Voornaam'
            break
        elif kolom == '2':
            kolom_naam='Achternaam'
            break
        elif kolom == '3':
            kolom_naam='Beroep'
            break
        elif kolom == '4':
            kolom_naam='Woonplaats'
            break
        elif kolom == '5':
            kolom_naam='Provincie'
            break
        elif kolom == '6':
            kolom_naam='CryptoNaam'
            break
        elif kolom == '7':
            kolom_naam='AankoopJaar'
            break
        elif kolom == 's':
            break
        else:
            print("Ongeldige keuze. Probeer het nog eens.")

    # Voorzie de grafiek van de juiste y-naam en titel.
    plt.title(f'Top {top} van meest voorkomende {kolom_naam}', fontsize=15)
    plt.ylabel(kolom_naam, fontsize=15)

    # Tel het aantal unieke waardes in de opgegeven kolom
    counts = df[kolom_naam].value_counts()

    # Selecteer de top x van deze kolom
    top = counts.head(top)

    # Geef de grafiek weer
    top.plot(kind=grafiek_naam)

# Help
Ik heb een online help-file gemaakt om wat extra screenshots te kunnen gebruiken

In [ ]:
def help():
    webbrowser.open('https://github.com/22161392/Fitymi/blob/main/README.md')

# Run
Hier worden alle verschillende functies aangeroepen via een Menu.
Dit programma geeft het beste resultaat als het aantal ligt tussen de 500 en 1000.

In [ ]:
def main():

    # Maak het scherm leeg (clear screen)
    os.system('cls' if os.name == 'nt' else 'clear')

    land='nl_NL'
    naam='BingoBit'

    # Omdat tinker niet werkt op mijn Mac heb ik ervoor gekozen om "misbruikt" te maken van de Jupyter Input dialog om een toch iets van een grafisch menu te maken.
    while True:
        choice = input("Maak een keuze :"
                       """

                        1).  Maak een dataset en schrijf deze weg als een xlsx bestand
                        2).  Open de dataset in Excel en start PowerBI online
                        3).  Visualiseer de dataset met matplotlib
                        a).  Genereer automatisch een dataset van 1000 rijen
                        h).  Open de online help
                        s).  Stop het programma
                         """)

        if choice == '1':
            aantal = int(input('Hoeveel klanten moeten er gegenereerd worden? (1000 is optimaal voor PowerBI dataset)'))
            klant_data=genereer_klant_data(land, aantal)
            df=manipuleer_klant_data(klant_data, naam)
        elif choice == '2':
            if 'df' in locals():
                converteer_klant_data(df, naam)
            else:
                print("Het DataFrame is leeg. Genereer eerst een dataset (optie 1 of a)")
        elif choice == '3':
            if 'df' in locals():
                visualiseer_klant_data(df)
                break
            else:
                print("Het DataFrame is leeg. Genereer eerst een dataset (optie 1 of a)")
        elif choice == 'a':
            klant_data=genereer_klant_data(land, aantal=1000)
            df=manipuleer_klant_data(klant_data, naam)
            visualiseer_klant_data(df)
            break
        elif choice == 'h':
            help()
        elif choice == 's':
            break
        else:
            print("Ongeldige keuze. Probeer het nog eens.")

if __name__ == '__main__':
    main()